<a href="https://colab.research.google.com/github/ELehmann91/NLP-Contract-Analysis/blob/master/gen_csv_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
!pip install PyPDF2

In [0]:
import os
import json
import  pandas as pd
import PyPDF2

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path = '/content/gdrive/Shared drives/FS-AI Base Data/innolytiq_pdf/'
file_list = os.listdir(path)
len(file_list)

149

In [0]:
name_list = []

with open(path + file_list[0]+'/data.json', encoding="utf8") as json_file:
    data = json.load(json_file)
    for line in data['output']:
        try: #line['members'] is not None:
            name_list.append(line['name'])
        except:
            print(line)

In [0]:
df_ = pd.DataFrame(index=file_list, columns=name_list)
df_ = df_.fillna(0) # with 0s rather than NaNs

In [0]:
for file in file_list:
    with open(path+file+'/data.json', encoding="utf8") as json_file:
        data = json.load(json_file)
        #print(data)
        try:
            for line in data['output']:
                try: # line['members'] is not None:
                    df_.loc[df_.index==file,line['name']] = line['data'][0]['value']
                except:
                    pass
                    #print(line['name'])
        except KeyError:
            print(file)

In [0]:
df_['pdf_text'] = 'leer'
for file in file_list:
  if file != 'series_1513__tranche_11.pdf':
    with open(path+file+'/'+file+'.pdf', 'rb') as pdfFileObj:
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        try:
            print(file, pdfReader.numPages)
            pdf_text = ' '.join([pdfReader.getPage(i).extractText() for i in range(0,min(pdfReader.numPages,2))])
            df_['pdf_text'][df_.index==file] = pdf_text.replace('\n',' ')
        except:
          print('cant read '+file)

erfurter_bank_eg.pdf 1


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [ipykernel_launcher.py:9]


de000dr0q3k7_en.pdf 2


cant read xf0000056977_-_fresenius_se.pdf
xf7000001779.pdf 1
xf0000053353_-_bayern_lb_nasv.pdf 2
xf7000002049.pdf 2
xf0000056184_-_sparkassenverband_b.w..pdf 4
xs1191881306.pdf 1
eurohypo_ag_oenapf.pdf 3
de0007572711_dt.pdf 7
de000de04xk4_dt_pib.pdf 3
de000hsh3j46_dt.pdf 1
de000a0fag76_dt.pdf 15
xf7000001879.pdf 1
xf7000001874.pdf 1
xf7000001950.pdf 1
de000a1h3v53_dt.pdf 90
de000dr9rhd9_dt.pdf 2
de000dr0q169_de.pdf 2
amba-#7711630-v1-executed_-_nwb_-_sdg_final_terms.pdf 7
xf7000002042.pdf 2
de000dr5dq47_eng.pdf 2
de000a0e83l5_dt.pdf 24
de000dr0rvp3_en.pdf 1
de000dr4wdy5_eng.pdf 2
de000dr9rhd9_en.pdf 2
final_terms_eur250m_series_1516.pdf 7
final_terms_zero_coupon_july2020.pdf 5
final_terms_eur85m_series_1445xxiii.pdf 8
final_terms_xs2066631206_series_1522.pdf 7
final_terms_series_1411_tranche_6.pdf 7
int_925013.pdf 1
final-terms-series-60-i-tranch2.pdf 9
icm-#33492791-v1-executed_final_terms_-_nwb_aug_2021.pdf 9
serie_1513_viii_final_terms_signed.pdf 8
xf0000056128_-_liebherr.pdf 12
xf0

In [0]:
list(df_)

['006_issue_date',
 '009_nominal_amount',
 '008_maturity_date',
 '010_redemption_formula',
 '011_seniority',
 '012_business_day_convention',
 '014_business_day_calendar',
 '017_first_payment_date',
 '018_payment_dates',
 '021_formula',
 '002_issuer_name',
 '020_paragraph_489',
 '004_identifiers',
 '007_issue_price',
 '003_product_subclass',
 '015_payment_frequency',
 '005_currency',
 '013_day_count_convention',
 '001_type',
 '004_wkn',
 '004_isin',
 '023_amendment',
 '022_legislation',
 'pdf_text']

In [0]:
classification = ['001_type','003_product_subclass','005_currency','011_seniority','012_business_day_convention','013_day_count_convention','014_business_day_calendar','015_payment_frequency','022_legislation','023_amendment']

In [0]:
len(df_[df_['015_payment_frequency'] == '1'])

109

In [0]:
df_bert = df_['pdf_text']

for var in classification:
  temp = pd.get_dummies(df_[var],prefix=var)
  for col in temp.columns:
    if sum(temp[col])<5:
      temp = temp.drop(col, axis=1)

  df_bert = pd.concat([df_bert,temp],axis=1)


In [0]:
df_bert[:2]

,pdf_text,001_type_Fixed,001_type_Floating,003_product_subclass_,003_product_subclass_Bond,003_product_subclass_Covered Bond,003_product_subclass_Schuldschein,003_product_subclass_other Loan,005_currency_,005_currency_EUR,005_currency_Euro,005_currency_Euro (€),005_currency_USD,011_seniority_,011_seniority_Senior,011_seniority_Senior Secured,011_seniority_Senior Unsecured,012_business_day_convention_,012_business_day_convention_Following adjusted,012_business_day_convention_Following unadjusted,012_business_day_convention_Modified following adjusted,012_business_day_convention_Unadjusted,013_day_count_convention_,013_day_count_convention_30/360,013_day_count_convention_301360,013_day_count_convention_Act/360,013_day_count_convention_Act/act ICMA,013_day_count_convention_Actual/Actual (ICMA),013_day_count_convention_act/act,014_business_day_calendar_,014_business_day_calendar_TARGET,014_business_day_calendar_TARGET2,015_payment_frequency_,015_payment_frequency_0,015_payment_frequency_1,015_payment_frequency_2,015_payment_frequency_4,022_legislation_,022_legislation_Ger,022_legislation_German Law,022_legislation_NLD,022_legislation_Netherlands,023_amendment_,023_amendment_no_amendment
erfurter_bank_eg.pdf,"1. Erfurt, den 19.02.2013 Unterschrift; SCHULD...",1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1
de000dr0q3k7_en.pdf,Dresdner Bank AG Private Investor Products +...,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1


In [0]:
df_bert.to_csv('/content/gdrive/Shared drives/FS-AI Base Data/df_bert.csv')